In [698]:
import os
import pandas as pd
import numpy as np
import nltk
import string
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import col

In [699]:
spark = SparkSession\
        .builder\
        .master('local[*]')\
        .appName('MvT')\
        .getOrCreate()

In [700]:
blog = spark.read.csv('/spring2021/project1/blogtext.csv', header = True, inferSchema=True)

In [701]:
hack = spark.read.csv('/spring2021/project1/hacker_news_sample.csv', header=True, inferSchema=True)

In [702]:
f1 = open("/spring2021/project1/comparison/Charles Dickens - Cities.txt", 'r')

In [703]:
f2 = open("/spring2021/project1/comparison/Conan Doyle - Sherlock.txt", 'r')

In [684]:
f3 = open("/spring2021/project1/comparison/Herman Melville - Moby.txt", 'r')

In [670]:
f4 = open("/spring2021/project1/comparison/Jane Austen - Pride.txt", 'r')

In [671]:
f5 = open("/spring2021/project1/comparison/Mary Shelley - Frankenstein.txt", 'r')

In [672]:
f6 = open("/spring2021/project1/comparison/Nathaniel Hawthorne - Scarlet.txt", 'r')

In [673]:
f7 = open("/spring2021/project1/comparison/Scott Fitzgerald - Gatsby.txt", 'r')

**Calculating Lexical Richness for Traditional Texts**

In [140]:
tradRaws = [f.read() for f in [f1, f2, f3, f4, f5, f6, f7]]

def returnLexical(raw):
    # Clean Raw
    startin = raw.find(" ***") + 4
    endin = raw.rfind("End of the Project Gutenberg")
    raw = raw[startin:endin]
    # tokenize
    tokens = nltk.word_tokenize(raw)
    text = nltk.Text(tokens)
    words = [w.lower() for w in text]
    vocab = len(set(words))
    length = len(words)
    return (vocab/length)

tradFDs = [returnLexical(raw) for raw in tradRaws]

In [141]:
#Average Lexical Richness
print(sum(tradFDs)/len(tradFDs))

0.07948531416423567


**Calculating Lexical Richness for Blog Texts**

In [705]:
blogRDD = blog.select('text').rdd

In [659]:
def returnLexical2(x):
    raw = x['text']
    tokens = nltk.word_tokenize(raw)
    text = nltk.Text(tokens)
    words = [w.lower() for w in text]
    vocab = len(set(words))
    length = len(words)
    #score=vocab/length
    return vocab


In [687]:
scoreRDD = blogRDD.map(returnLexical2)

In [706]:
from operator import add
def returnLexical2(x):
    raw = x['text']
    tokens = nltk.word_tokenize(raw)
    text = nltk.Text(tokens)
    words = [w.lower() for w in text]
    score = 0
    length=1
    vocab=0
    for word in words:
        length+=1
    for word in set(words):
        vocab+=1
    score=vocab/length
    return score

scoreRDD = blogRDD.map(returnLexical2)

In [707]:
cumulScore = scoreRDD.reduce(add)
cumulScore

430918.9636495318

In [708]:
def avgMap(x):
    return (x,1)

def avgRed(x,y):
    return (x[0] + y[0], x[1] + y[1])

avgRDD = scoreRDD.map(avgMap)

avgTup = avgRDD.reduce(avgRed)


avg = avgTup[0]/avgTup[1]

avg
#240.18463078539932
#116.02464170595522
#0.4833

0.6325100305445773

**Calculating Lexical Richness for Blog Texts**

In [644]:
hackRDD = hack.select('text').rdd

In [645]:
from bs4 import BeautifulSoup
def returnLexical3(x):
    if x['text'] != None:
        bs = BeautifulSoup(x['text'], 'lxml')
        raw = bs.get_text()
        tokens = nltk.word_tokenize(raw)
        text = nltk.Text(tokens)
        words = [w.lower() for w in text]
        vocab = len(set(words))
        length = len(words)
        score=vocab/length
    else:
        score=0
    return score


In [646]:
scoreHackRDD = hackRDD.map(returnLexical3)

In [647]:
cumulScore2 = scoreHackRDD.reduce(add)
cumulScore2

134692.215181225

In [648]:
def avgMap(x):
    return (x,1)

def avgRed(x,y):
    return (x[0] + y[0], x[1] + y[1])

avgHackRDD = scoreHackRDD.map(avgMap)

avgHackTup = avgHackRDD.reduce(avgRed)

avg = avgHackTup[0]/avgHackTup[1]

avg

#0.6262802530431215

0.6262802530431215

In [649]:
spark.stop()